In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('all_features_2.csv', index_col= None)
df = df.drop('Unnamed: 0', axis=1)
# randomize the data
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,Width,Height,H_W_Ratio,Jaw_width,J_F_Ratio,MJ_width,MJ_J_width,filenum,filename,classified_shape
0,70.0,93.0,72.0,114.0,75.0,134.0,78.0,156.0,86.0,175.0,...,155.003226,206.000000,1.329005,69.007246,0.445199,126.035709,1.826413,31,brittany snow 15.jpg,heart
1,70.0,75.0,74.0,99.0,78.0,121.0,84.0,143.0,92.0,163.0,...,157.003185,192.000000,1.222905,65.000000,0.414004,113.000000,1.738462,832,Katy Perry (17).jpg,oval
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1253,Kate Upton (3).jpg,round
3,77.0,96.0,78.0,116.0,82.0,136.0,88.0,156.0,97.0,173.0,...,136.014705,202.158354,1.486298,54.147945,0.398104,95.084173,1.756007,1009,Naomi Campbell (6).jpg,long
4,67.0,96.0,68.0,117.0,71.0,138.0,72.0,160.0,79.0,180.0,...,150.029997,216.231358,1.441254,72.173402,0.481060,128.062485,1.774372,883,allison williams 3.jpg,oval


In [3]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import ensemble
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.decomposition import PCA 

In [4]:
data = df.dropna(axis=0, how='any')
X = data
X = X.drop(['filenum','filename','classified_shape'] , axis = 1)
X_norm = normalize(X)
Y = data['classified_shape']
Y.head()

0    heart
1     oval
3     long
4     oval
5    round
Name: classified_shape, dtype: object

In [5]:
scaler = StandardScaler()  
scaler.fit(X)  

X = scaler.transform(X)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y,
    test_size=0.25,
    random_state=1200)

In [7]:
n_components = 18
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [8]:
# #Remove PCA 
X_train_pca = X_train
X_test_pca = X_test

In [9]:
# train a LDA classifier and print the accuracy and confusion matrix
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_pca, Y_train)
Y_pred = lda.predict(X_test_pca)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))


Accuracy: 0.6266666666666667
[[37  4 17  5  0]
 [ 5 45 17  2  0]
 [ 9 16 59 10  1]
 [ 7  0 12 47  0]
 [ 1  3  2  1  0]]
              precision    recall  f1-score   support

       heart       0.63      0.59      0.61        63
        long       0.66      0.65      0.66        69
        oval       0.55      0.62      0.58        95
       round       0.72      0.71      0.72        66
      square       0.00      0.00      0.00         7

    accuracy                           0.63       300
   macro avg       0.51      0.51      0.51       300
weighted avg       0.62      0.63      0.62       300



In [10]:
# train a MLP classifier and print the accuracy and confusion matrix
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
mlp.fit(X_train_pca, Y_train)
Y_pred = mlp.predict(X_test_pca)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

Accuracy: 0.6433333333333333
[[41  5 13  4  0]
 [ 4 49 12  2  2]
 [21 11 53  8  2]
 [ 8  1  8 49  0]
 [ 0  2  3  1  1]]
              precision    recall  f1-score   support

       heart       0.55      0.65      0.60        63
        long       0.72      0.71      0.72        69
        oval       0.60      0.56      0.58        95
       round       0.77      0.74      0.75        66
      square       0.20      0.14      0.17         7

    accuracy                           0.64       300
   macro avg       0.57      0.56      0.56       300
weighted avg       0.64      0.64      0.64       300



In [11]:
# save the lda model to disk 

import pickle
filename = 'mlp_model.sav'
pickle.dump(mlp, open(filename, 'wb'))

In [12]:
from conversion_to_data_frame import *
test_photo = 'test_photo.jpg'
df = pd.DataFrame(columns = ['0','1','2','3','4','5','6','7','8','9','10','11',	'12',	'13',	'14',	'15',	'16','17',
                             '18',	'19',	'20',	'21',	'22',	'23',	'24','25',	'26',	'27',	'28',	'29',
                             '30',	'31',	'32',	'33',	'34',	'35',	'36',	'37',	'38',	'39',	'40',	'41',
                             '42',	'43',	'44',	'45',	'46',	'47',	'48',	'49',	'50',	'51',	'52',	'53',
                             '54',	'55',	'56',	'57',	'58',	'59',	'60',	'61',	'62',	'63',	'64',	'65',
                             '66',	'67',	'68',	'69',	'70',	'71',	'72',	'73',	'74',	'75',	'76',	'77',
                             '78',	'79',	'80',	'81',	'82',	'83',	'84',	'85',	'86',	'87',	'88',	'89',
                             '90',	'91',	'92',	'93',	'94',	'95',	'96',	'97',	'98',	'99',	'100',	'101',
                             '102',	'103',	'104',	'105',	'106',	'107',	'108',	'109',	'110',	'111',	'112',	'113',
                             '114',	'115',	'116',	'117',	'118',	'119',	'120',	'121',	'122',	'123',	'124',	'125',
                             '126',	'127',	'128',	'129',	'130',	'131',	'132',	'133',	'134',	'135',	'136',	'137',
                             '138',	'139',	'140',	'141',	'142',	'143','A1','A2','A3','A4','A5','A6','A7','A8','A9',
                            'A10','A11','A12','A13','A14','A15','A16','Width','Height','H_W_Ratio','Jaw_width','J_F_Ratio',
                             'MJ_width','MJ_J_width'])

style_df = pd.DataFrame()
style_df = pd.DataFrame(columns = ['face_shape','hair_length','location','filename','score'])
make_face_df_save(test_photo,2305,df)
df.head()


[[[102  58  34]
  [102  58  34]
  [102  58  34]
  ...
  [148 118  78]
  [148 118  78]
  [148 118  78]]

 [[102  58  34]
  [102  58  34]
  [102  58  34]
  ...
  [148 118  78]
  [148 118  78]
  [148 118  78]]

 [[102  58  34]
  [102  58  34]
  [102  58  34]
  ...
  [148 118  78]
  [148 118  78]
  [148 118  78]]

 ...

 [[ 90  50  27]
  [ 90  50  27]
  [ 90  50  27]
  ...
  [205 140 108]
  [207 141 109]
  [207 141 109]]

 [[ 90  50  27]
  [ 90  50  27]
  [ 90  50  27]
  ...
  [205 141 106]
  [207 142 107]
  [207 142 107]]

 [[ 90  50  27]
  [ 90  50  27]
  [ 90  50  27]
  ...
  [205 141 106]
  [207 142 107]
  [207 142 107]]]
[{'chin': [(163, 475), (168, 507), (179, 540), (195, 571), (213, 603), (234, 633), (261, 661), (292, 682), (332, 682), (376, 668), (418, 641), (457, 609), (487, 571), (504, 526), (508, 475), (504, 422), (495, 372)], 'left_eyebrow': [(145, 421), (153, 404), (169, 395), (191, 392), (213, 396)], 'right_eyebrow': [(259, 372), (288, 349), (321, 333), (358, 330), (396, 339)

,0,1,2,3,4,5,6,7,8,9,...,A14,A15,A16,Width,Height,H_W_Ratio,Jaw_width,J_F_Ratio,MJ_width,MJ_J_width
2305,92.0,106.0,89.0,122.0,89.0,139.0,91.0,157.0,94.0,174.0,...,59.111534,52.431408,46.468801,167.146642,236.211769,1.413201,77.646635,0.464542,137.116739,1.765907


In [13]:
#load the model from disk and than predict the face shape

loaded_model = pickle.load(open('mlp_model.sav', 'rb'))
result = loaded_model.predict(df)
print(result)



['long']


C:\Users\qayse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
